Code to calculate the orbital evolution of Phobos and produce Figure 1 of the paper :

a) We solve for a,e of Phobos as a function of time using high-order tidal evolution equations 

b) Keep track of the velocity of the object - (r,v_r,v_theta,v_total)


In [1]:
%pylab

Using matplotlib backend: TkAgg
Populating the interactive namespace from numpy and matplotlib


In [10]:
from astropy import constants
from scipy import integrate

G_val = constants.G.si.value # Gravitational Constant

M_mars = 6.41693*(10**23.) # Mars mass, kg
R_mars = 3389.5*1e3        # Mars, m
rho_mars = 3933            # kg/m^3, Mars density
V_phobos = 5689*1e9        # +/- 60*1e9 m^3, Phobos Volume
M_phobos = 1.065*(10**16.) # Phobos Mass, +/- .015 kg
R_phobos = 11.3*1e3        # Phobos Radius, m
rho_phobos = 1860.         # Phobos density, +/- 31 kg/m^3
ecc_phobos =  0.01511      # Current eccentricity of Phobos
a_phobos = 9378*1e3        # Semi-major Axis of Phobos, m 
w_phobos = 1./27562.       # 1/Rotation period of phobos ~ 7hr 39 mins, 1/s

k2_mars = .148            # In Nimmo 2013; .148 +/- .017 ; Other Sets in Jacobsen 2014 - 0.1837 +/- .0009
Q_mars = 88.              # In Nimmo 2013 88. +/- 16, 170+/- 20 ; Other Sets in Jacobsen 2014 - 99.57 +/- 4.9 

mu_phobos_monolith = 8e4  # Dimensionless, from 8e4 - 8e6
mu_phobos_rubble = sqrt(mu_phobos_monolith/1e-2) # Dimensionless, ~ sqrt(mu/epsilon_yield_strain); ~ from 2.85e3 - 2.85e4
k2_phobos = 1.5/(1. + mu_phobos_rubble)
Q_phobos = 100.                         # Can be between 50 - 100

# Some terms for the tidal evolution equation (See text for details)

mu_phobos_C1 = (4.*pi/19.)*(rho_phobos**2.)*(R_phobos**2.)*G_val/k2_phobos
const_Ecc1 = -(57./8.)*(k2_mars/Q_mars)*sqrt(G_val/M_mars)*(R_mars**5.)*M_phobos
const_Ecc2 = -(21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(M_mars/M_phobos)

kappa = (rho_phobos/rho_mars)*(R_phobos/R_mars)**3.
rad_ratio = R_phobos/R_mars
    
mu_mars_C1 = (4.*pi/19.)*(rho_mars**2.)*(R_mars**2.)*G_val/k2_mars
const_a1 = (8.*sqrt(3)/19.)*(pi*G_val)**1.5*(R_mars**2.)*(rho_mars**2.5)*kappa*sqrt(1.+kappa)*(R_mars**5.5)/mu_mars_C1/Q_mars
    
def dX_dt(t,X):
    """ X is [a,e] - for moon/Phobos, a is in units of R_mars """
    dt_e = (const_Ecc1 + const_Ecc2)*(X[1])/((X[0]*R_mars)**6.5)
    w_t1 = + (19./22.)*(rad_ratio**2.)/(X[0]**2.)
    w_t2 = + (380./459.)*(rad_ratio**4.)/(X[0]**4.)
    w_t3 = + (475./584.)*(rad_ratio**6.)/(X[0]**6.)
    w_t4 = + (133./165.)*(rad_ratio**8.)/(X[0]**8.)
    a_t1 = + (19./22.)/(X[0]**2.)
    a_t2 = + (380./459.)/(X[0]**4.)
    a_t3 = + (475./584.)/(X[0]**6.)
    a_t4 = + (133./165.)/(X[0]**8.)
    dt_a = -1.*(const_a1/((X[0]*R_mars)**5.5))*(1. + 51.*X[1]*X[1]/4.)*(1. + a_t1 + a_t2 + a_t3 + a_t4) # Is really dt_a/R_mars
    if (X[1] <=0.): # Prevent non-zero eccentricity
        dt_e =0.
    if (X[0] <=1.): # Stop evolution once we reach the top of the Martian atmosphere
        dt_a =0.
        dt_e =0.
    return array([dt_a,dt_e])

# Integrate the orbit 

max_t = 80.*1e6*3.154e7                                         # Total integration time, unit is per seconds
dt = 1e3*3.154e7                                                # time-step   
X_init = array([a_phobos/R_mars,ecc_phobos])                    # initials conditions: [a,e]
abc = integrate.ode(dX_dt).set_integrator('vode',nsteps=1e8)
abc.set_initial_value(X_init,0.0)

t_vX=[]
a_intX=[]
e_intX=[]

a_intX.append(X_init[0])
e_intX.append(X_init[1])
t_vX.append(0.)

while abc.t < max_t :
	#print abc.t/max_t
	X = abc.integrate(abc.t+dt)
	a_intX.append(X[0])
	e_intX.append(X[1])
	t_vX.append(abc.t)
	#print abc.t/max_t
    
a_int = array(a_intX)      # Orbital semi-major axis
e_int = array(e_intX)      # Orbital eccentricity
t_v = array(t_vX)          # time

r_peri = a_int*(1. - e_int) # periapsis location 
r_apo  = a_int*(1. + e_int) # apoapsis location 

Vel_r_max = sqrt(G_val*M_mars*(2./r_apo- 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s
Vel_r_min = sqrt(G_val*M_mars*(2./r_peri - 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s

# Plotting section ... 

import numpy as np
import pylab as plt
from math import asin

fig_width_pt = 2.*130.0                 # Get this from LaTeX using \showthe\columnwidth
inches_per_pt = 1.0/72.27               # Convert pt to inch
golden_mean = (sqrt(5)-1.0)/2.0         # Aesthetic ratio
fig_width = fig_width_pt*inches_per_pt  # width in inches
fig_height = 1.3*fig_width*golden_mean  # height in inches
params = {'axes.labelsize': 10,
          'text.fontsize': 8,
          'text.fontweight': 30,
          'legend.fontsize': 8,
          'xtick.labelsize': 6.,
          'axes.linewidth': .3,
          'ytick.labelsize': 6.}
pylab.rcParams.update(params)

import matplotlib.gridspec as gridspec
fig, axes = plt.subplots(nrows=3, ncols=1, sharex=True, sharey=True,figsize=(fig_width,fig_width*.8),dpi=250)
gs1 = gridspec.GridSpec(3, 1)
gs1.update(wspace=0.05, hspace=0.00005) # set the spacing between axes. 

ax = plt.subplot(gs1[0])
ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,a_int[a_int>1.+160./R_mars],'k-',linewidth=1.)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=False)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ax.set_ylabel('a/R$_{Mars}$',labelpad=11)
ylim([.8,6.99])

ax = plt.subplot(gs1[1])

ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,e_int[a_int>1.+160./R_mars],'k-',linewidth=1.)
ax.get_yaxis().get_major_formatter().set_powerlimits((-4,6))
ax.get_yaxis().get_offset_text().set_x(-.17)
ax.get_yaxis().get_offset_text().set_y(-.5)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=False)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ax.set_ylabel('$e$',labelpad=4.5)
ylim([-.001,0.017])

ax = plt.subplot(gs1[2])
ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,Vel_r_max[a_int>1.+160./R_mars]/1e3,'k-',linewidth=1.)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=True)
ax.set_xlabel("$t$ (Myr)")

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ax.set_ylabel('V (km/s)',labelpad=9.5)
ylim([1.95,3.7])

plt.gcf().subplots_adjust(bottom=0.15)
plt.gcf().subplots_adjust(left=.15)

/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontsize is deprecated and replaced with font.size; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))
/usr/lib/python3/dist-packages/matplotlib/__init__.py:857: UserWarning: text.fontweight is deprecated and replaced with font.weight; please use the latter.
  warnings.warn(self.msg_depr % (key, alt_key))


In [16]:
# In this section, the same code as above is repeated with different values
# of Mars' Q and k_2 to get the three lines in Figure 1

from scipy import integrate

k2_mars = .1463  # In Nimmo 2013; .148 +/- .017 ; Other Sets in Jacobsen 2014 - 0.1837 +/- .0009
Q_mars =   190.    # In Nimmo 2013 88. +/- 16, 170+/- 20 ; Other Sets in Jacobsen 2014 - 99.57 +/- 4.9 

mu_phobos_rubble = sqrt(mu_phobos_monolith/1e-2) # Dimensionless, ~ sqrt(mu/epsilon_yield_strain); ~ from 2.85e3 - 2.85e4
k2_phobos = 1.5/(1. + mu_phobos_rubble)
Q_phobos = 100. # Can be between 50 - 100

mu_phobos_C1 = (4.*pi/19.)*(rho_phobos**2.)*(R_phobos**2.)*G_val/k2_phobos
const_Ecc1 = -(57./8.)*(k2_mars/Q_mars)*sqrt(G_val/M_mars)*(R_mars**5.)*M_phobos
const_Ecc2 = -(21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(M_mars/M_phobos)

kappa = (rho_phobos/rho_mars)*(R_phobos/R_mars)**3.
rad_ratio = R_phobos/R_mars
    
mu_mars_C1 = (4.*pi/19.)*(rho_mars**2.)*(R_mars**2.)*G_val/k2_mars
const_a1 = (8.*sqrt(3)/19.)*(pi*G_val)**1.5*(R_mars**2.)*(rho_mars**2.5)*kappa*sqrt(1.+kappa)*(R_mars**5.5)/mu_mars_C1/Q_mars
    
def dX_dt(t,X):
    """ X is [a,e] - for moon/Phobos, a is in units of R_mars """
    dt_e = (const_Ecc1 + const_Ecc2)*(X[1])/((X[0]*R_mars)**6.5)
    w_t1 = + (19./22.)*(rad_ratio**2.)/(X[0]**2.)
    w_t2 = + (380./459.)*(rad_ratio**4.)/(X[0]**4.)
    w_t3 = + (475./584.)*(rad_ratio**6.)/(X[0]**6.)
    w_t4 = + (133./165.)*(rad_ratio**8.)/(X[0]**8.)
    a_t1 = + (19./22.)/(X[0]**2.)
    a_t2 = + (380./459.)/(X[0]**4.)
    a_t3 = + (475./584.)/(X[0]**6.)
    a_t4 = + (133./165.)/(X[0]**8.)
    dt_a = -1.*(const_a1/((X[0]*R_mars)**5.5))*(1. + 51.*X[1]*X[1]/4.)*(1. + a_t1 + a_t2 + a_t3 + a_t4) # Is really dt_a/R_mars
    if (X[1] <=0.):
        dt_e =0.
    if (X[0] <=1.):
        dt_a =0.
        dt_e =0.
        dt_w =0.
        dt_E =0.
    return array([dt_a,dt_e])

# Begin integration 
max_t =80.*1e6*3.154e7                            # unit is per seconds
dt = 1e3*3.154e7                                  # unit is per seconds
X_init = array([a_phobos/R_mars,ecc_phobos])               # initials conditions: [a,e]
abc = integrate.ode(dX_dt).set_integrator('vode',nsteps=1e8)
abc.set_initial_value(X_init,0.0)

t_vX=[]
a_intX=[]
e_intX=[]

a_intX.append(X_init[0])
e_intX.append(X_init[1])
t_vX.append(0.)

while abc.t < max_t :
	#print abc.t/max_t
	X = abc.integrate(abc.t+dt)
	a_intX.append(X[0])
	e_intX.append(X[1])
	t_vX.append(abc.t)
    
a_int = array(a_intX)
e_int = array(e_intX)
t_v = array(t_vX)

r_peri =a_int*(1. - e_int) # periapse location 
r_apo = a_int*(1. + e_int) # apoapse location 
Vel_r_max = sqrt(G_val*M_mars*(2./r_apo- 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s
Vel_r_min = sqrt(G_val*M_mars*(2./r_peri - 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s

# Begin plotting ...

ax = plt.subplot(gs1[0])
ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,a_int[a_int>1.+160./R_mars],'b-',linewidth=.6)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=False)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ylim([.8,2.99])

ax = plt.subplot(gs1[1])

ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,e_int[a_int>1.+160./R_mars],'b-',linewidth=.6)
ax.get_yaxis().get_major_formatter().set_powerlimits((-4,6))
ax.get_yaxis().get_offset_text().set_x(-.17)
ax.get_yaxis().get_offset_text().set_y(-.5)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=False)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ylim([-.001,0.017])

ax = plt.subplot(gs1[2])
ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,Vel_r_max[a_int>1.+160./R_mars]/1e3,'b-',linewidth=.6)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=True)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ylim([1.95,3.7])


######################  3rd set of k2 and Q for Mars


from scipy import integrate

k2_mars = .1497  # In Nimmo 2013; .148 +/- .017 ; Other Sets in Jacobsen 2014 - 0.1837 +/- .0009
Q_mars = 72.    # In Nimmo 2013 88. +/- 16, 170+/- 20 ; Other Sets in Jacobsen 2014 - 99.57 +/- 4.9 

mu_phobos_rubble = sqrt(mu_phobos_monolith/1e-2) # Dimensionless, ~ sqrt(mu/epsilon_yield_strain); ~ from 2.85e3 - 2.85e4
k2_phobos = 1.5/(1. + mu_phobos_rubble)
Q_phobos = 100. # Can be between 50 - 100

mu_phobos_C1 = (4.*pi/19.)*(rho_phobos**2.)*(R_phobos**2.)*G_val/k2_phobos
const_Ecc1 = -(57./8.)*(k2_mars/Q_mars)*sqrt(G_val/M_mars)*(R_mars**5.)*M_phobos
const_Ecc2 = -(21./2.)*(k2_phobos/Q_phobos)*sqrt(G_val*M_mars)*(R_phobos**5.)*(M_mars/M_phobos)

kappa = (rho_phobos/rho_mars)*(R_phobos/R_mars)**3.
rad_ratio = R_phobos/R_mars
    
mu_mars_C1 = (4.*pi/19.)*(rho_mars**2.)*(R_mars**2.)*G_val/k2_mars
const_a1 = (8.*sqrt(3)/19.)*(pi*G_val)**1.5*(R_mars**2.)*(rho_mars**2.5)*kappa*sqrt(1.+kappa)*(R_mars**5.5)/mu_mars_C1/Q_mars
    
def dX_dt(t,X):
    """ X is [a,e,w_s,E] - for moon/Phobos , E is the Energy, a is in units of R_mars """
    dt_e = (const_Ecc1 + const_Ecc2)*(X[1])/((X[0]*R_mars)**6.5)
    w_t1 = + (19./22.)*(rad_ratio**2.)/(X[0]**2.)
    w_t2 = + (380./459.)*(rad_ratio**4.)/(X[0]**4.)
    w_t3 = + (475./584.)*(rad_ratio**6.)/(X[0]**6.)
    w_t4 = + (133./165.)*(rad_ratio**8.)/(X[0]**8.)
    a_t1 = + (19./22.)/(X[0]**2.)
    a_t2 = + (380./459.)/(X[0]**4.)
    a_t3 = + (475./584.)/(X[0]**6.)
    a_t4 = + (133./165.)/(X[0]**8.)
    dt_a = -1.*(const_a1/((X[0]*R_mars)**5.5))*(1. + 51.*X[1]*X[1]/4.)*(1. + a_t1 + a_t2 + a_t3 + a_t4) # Is really dt_a/R_mars
    if (X[1] <=0.):
        dt_e =0.
    if (X[0] <=1.):
        dt_a =0.
        dt_e =0.
        dt_w =0.
        dt_E =0.
    return array([dt_a,dt_e])

# Begin integration ...

max_t =80.*1e6*3.154e7                            # unit is per seconds
dt = 1e2*3.154e7                                  # unit is per seconds
X_init = array([a_phobos/R_mars,ecc_phobos])               # initials conditions: [a,e]
abc = integrate.ode(dX_dt).set_integrator('vode',nsteps=1e8)
abc.set_initial_value(X_init,0.0)

t_vX=[]
a_intX=[]
e_intX=[]

a_intX.append(X_init[0])
e_intX.append(X_init[1])
t_vX.append(0.)

while abc.t < max_t :
	#print abc.t/max_t
	X = abc.integrate(abc.t+dt)
	a_intX.append(X[0])
	e_intX.append(X[1])
	t_vX.append(abc.t)
    
a_int = array(a_intX)
e_int = array(e_intX)
t_v = array(t_vX)

r_peri =a_int*(1. - e_int) # periapse location 
r_apo = a_int*(1. + e_int) # apoapse location 
Vel_r_max = sqrt(G_val*M_mars*(2./r_apo- 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s
Vel_r_min = sqrt(G_val*M_mars*(2./r_peri - 1./a_int)/R_mars)# Total Velocity at the periapse for the given a, m/s
    
# Begin plotting ..

ax = plt.subplot(gs1[0])
ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,a_int[a_int>1.+160./R_mars],'r-',linewidth=.6)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=False)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ylim([.8,2.99])

ax = plt.subplot(gs1[1])

ax.plot(t_v[a_int>1.+160./R_mars]/3.154e7/1e6,e_int[a_int>1.+160./R_mars],'r-',linewidth=.6)
ax.get_yaxis().get_major_formatter().set_powerlimits((-4,6))
ax.get_yaxis().get_offset_text().set_x(-.17)
ax.get_yaxis().get_offset_text().set_y(-.5)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=False)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ylim([-.001,0.017])

ax = plt.subplot(gs1[2])
ax.plot(t_v[a_int>1.+158./R_mars]/3.154e7/1e6,Vel_r_max[a_int>1.+158./R_mars]/1e3,'r-',linewidth=.6)

xticklabels = ax.get_xticklabels()
setp(xticklabels, visible=True)

yticklabels = ax.get_yticklabels()
setp(yticklabels, visible=True)
ylim([1.95,3.7])


ax = plt.subplot(gs1[1])
t = ax.text(45., 0.013,'$k_{2,Mars} = 0.148$; $Q_{Mars} = 88$',size=6,fontweight="bold",color='k')
t = ax.text(45., 0.010,'$k_{2,Mars} = 0.1463$; $Q_{Mars} = 190$',size=6,fontweight="bold",color='b')
t = ax.text(45., 0.007,'$k_{2,Mars} = 0.1497$; $Q_{Mars} = 72$',size=6,fontweight="bold",color='r')

from matplotlib.patches import Rectangle
ax = plt.subplot(gs1[1])
ax.add_patch(Rectangle((43.8, .0058), (75.3-42.8), 0.01,alpha=1,facecolor='none',linewidth=.5))
plt.show()

In [9]:
savefig("Fig1.pdf",dpi=1200)
#py.plot(arr[:,0], arr[:,1], 'o', alpha=0.1, rasterized=True)
#py.savefig('dots.pdf', dpi=400)
